In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import os
import tensorflow as tf
from tensorflow.keras import backend as K
from utility import LoadDataIot
from IOT_CNN import DFNet_Dropout  #DFNet_Add_Layer
import random
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.models import Model
import numpy as np
import pandas as pd

from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
from sklearn.utils import shuffle
from tensorflow.keras.models import Model
from tensorflow.python.ops.numpy_ops import np_config
from sklearn.metrics import accuracy_score
np_config.enable_numpy_behavior()


In [3]:
!python dataset/IoT_Class_filter.py

2024-02-14 07:17:52.818649: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-14 07:17:52.818706: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-14 07:17:52.820387: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-14 07:17:54.620411: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Data dimensions:
X: Training data's shape :  (41898, 475)
X: Validating data's shape :  (10774, 475)
X: Testing data's shape :  (7183, 475)
X: open data's shape :  (86886, 475)


In [4]:
NB_EPOCH = 40   # Number of training epoch
print ("Number of Epoch: ", NB_EPOCH)
BATCH_SIZE = 70 # Batch size
VERBOSE = 2 # Output display mode
LENGTH = 475 # Packet sequence length


OPTIMIZER = Adamax(learning_rate=0.05, beta_1=0.9, beta_2=0.999, epsilon=1e-08) # Optimizer

NB_CLASSES = 40 # number of outputs = number of classes
INPUT_SHAPE = (LENGTH,1)

random.seed(0)

def representative_data_gen():
  for input_value in tensorflow.data.Dataset.from_tensor_slices(X_train_Rep).batch(1).take(100):
    # Model has only one input so each data point has one element.
    yield [input_value]


def Micro_F1(matrix):
    epsilon = 1e-8
    TP = 0
    FP = 0
    TN = 0

    for k in range(NB_CLASSES):
        TP += matrix[k][k]
        FP += (np.sum(Matrix, axis=0)[k] - matrix[k][k])
        TN += (np.sum(Matrix, axis=1)[k] - matrix[k][k])

    Micro_Prec = TP / (TP + FP)
    Micro_Rec = TP / (TP + TN)
    print("Micro Precision: ", Micro_Prec)
    print("Micro Recall: ", Micro_Rec)
    Micro_F1 = 2 * Micro_Prec * Micro_Rec / (Micro_Rec + Micro_Prec + epsilon)

    return Micro_F1


def New_F1_Score(Matrix):
    Column_sum = np.sum(Matrix, axis=0)
    Raw_sum = np.sum(Matrix, axis=1)

    Precision_Differences = []
    Recall_Differences = []
    for i in range(NB_CLASSES):
        Precision_Differences.append(np.abs(2 * Matrix[i][i] - Column_sum[i]))
        Recall_Differences.append(np.abs(2 * Matrix[i][i] - Raw_sum[i]))

    Precision_Differences = np.array(Precision_Differences)
    Precision_Differences_Per = Precision_Differences / np.sum(Precision_Differences)
    Recall_Differences = np.array(Recall_Differences)
    Recall_Differences_Per = Recall_Differences / np.sum(Recall_Differences)

    # print('Precision_Differences_Per',Precision_Differences_Per)
    # print('Recall_Differences_Per',Recall_Differences_Per)

    Precisions = np.zeros(NB_CLASSES)
    Recalls = np.zeros(NB_CLASSES)

    epsilon = 1e-8

    for k in range(len(Precisions)):
        Precisions[k] = (Matrix[k][k] / np.sum(Matrix, axis=0)[k])
    Precision = np.sum(np.array(Precisions) * Precision_Differences_Per)

    for k in range(len(Recalls)):
        Recalls[k] = (Matrix[k][k] / np.sum(Matrix, axis=1)[k])  # *Recall_Differences_Per[k]
    Recall = np.sum(np.array(Recalls) * Recall_Differences_Per)

    print('Precision: ', Precision)
    print('Recall: ', Recall)

    F1_Score = 2 * Precision * Recall / (Precision + Recall + epsilon)
    return F1_Score


def Macro_F1(Matrix):
    Precisions = np.zeros(NB_CLASSES)
    Recalls = np.zeros(NB_CLASSES)

    epsilon = 1e-8

    for k in range(len(Precisions)):
        Precisions[k] = Matrix[k][k] / np.sum(Matrix, axis=0)[k]
    # print(Precisions)

    Precision = np.average(Precisions)
    print("Precision:", Precision)

    for k in range(len(Recalls)):
        Recalls[k] = Matrix[k][k] / np.sum(Matrix, axis=1)[k]

    Recall = np.average(Recalls)
    print("Recall:", Recall)

    F1_Score = 2 * Precision * Recall / (Precision + Recall + epsilon)
    return F1_Score

print ("Loading and preparing data for training, and evaluating the model")
X_train, y_train, X_valid, y_valid, X_test, y_test = LoadDataIot()
# Please refer to the dataset format in readme

# Convert data as float32 type
X_train = X_train.astype('float32')
X_valid = X_valid.astype('float32')
X_test = X_test.astype('float32')
y_train = y_train.astype('float32')
y_valid = y_valid.astype('float32')
y_test = y_test.astype('float32')
# we need a [Length x 1] x n shape as input to the DF CNN (Tensorflow)
X_train = X_train[:, :,np.newaxis]
X_valid = X_valid[:, :,np.newaxis]
X_test = X_test[:, :,np.newaxis]

print(X_train.shape[0], 'train samples')
print(X_valid.shape[0], 'validation samples')
print(X_test.shape[0], 'test samples')

# Convert class vectors to categorical classes matrices
y_train = to_categorical(y_train, NB_CLASSES)
y_valid = to_categorical(y_valid, NB_CLASSES)
y_test = to_categorical(y_test, NB_CLASSES)
# Building and training model
# print ("Building and training DF model")

model = DFNet_Dropout.build(input_shape=INPUT_SHAPE, nb_classes=NB_CLASSES)

model.compile(loss="categorical_crossentropy", optimizer=OPTIMIZER,
	metrics=["accuracy"])
print ("Model compiled")

filepath = '/content/trained_models/IoT.hdf5'

checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

# Start training
history = model.fit(X_train, y_train,
		batch_size=BATCH_SIZE, epochs=NB_EPOCH,
		verbose=VERBOSE, validation_data=(X_valid, y_valid), callbacks=callbacks_list)

# Start evaluating model with testing data
score_test = model.evaluate(X_test, y_test, verbose=VERBOSE)
print("Testing closed accuracy_without_norm:", score_test[1])

model_penultimate = Model(model.input, model.layers[-2].output)
model_penultimate.save('/content/trained_models/IoT_without_softmax.hdf5')

del model


Number of Epoch:  40
Loading and preparing data for training, and evaluating the model
Data dimensions:
X: Training data's shape :  (41898, 475)
y: Training data's shape :  (41898,)
X: Validation data's shape :  (10774, 475)
y: Validation data's shape :  (10774,)
X: Testing data's shape :  (7183, 475)
y: Testing data's shape :  (7183,)
41898 train samples
10774 validation samples
7183 test samples
Model compiled
Epoch 1/40


599/599 - 44s - loss: 1.5512 - accuracy: 0.5024 - val_loss: 1.1593 - val_accuracy: 0.6139 - 44s/epoch - 74ms/step
Epoch 2/40


599/599 - 32s - loss: 0.8477 - accuracy: 0.7308 - val_loss: 0.9125 - val_accuracy: 0.7111 - 32s/epoch - 54ms/step
Epoch 3/40


599/599 - 32s - loss: 0.5941 - accuracy: 0.8135 - val_loss: 0.9079 - val_accuracy: 0.7419 - 32s/epoch - 54ms/step
Epoch 4/40


599/599 - 32s - loss: 0.4651 - accuracy: 0.8542 - val_loss: 0.5246 - val_accuracy: 0.8432 - 32s/epoch - 54ms/step
Epoch 5/40


...

599/599 - 32s - loss: 0.0769 - accuracy: 0.9754 - val_loss: 0.1624 - val_accuracy: 0.9614 - 32s/epoch - 54ms/step
Epoch 39/40


599/599 - 31s - loss: 0.0760 - accuracy: 0.9762 - val_loss: 0.1306 - val_accuracy: 0.9673 - 31s/epoch - 53ms/step
Epoch 40/40


599/599 - 31s - loss: 0.0783 - accuracy: 0.9750 - val_loss: 0.1130 - val_accuracy: 0.9735 - 31s/epoch - 53ms/step
225/225 - 2s - loss: 0.1093 - accuracy: 0.9708 - 2s/epoch - 9ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Testing closed accuracy_without_norm: 0.9707642793655396


In [5]:
################## Calculating mean anchor vectors for each class ######################

tflite_model_predictions = model_penultimate.predict(X_train)

y_train= np.argmax(y_train,axis=1)
y_valid= np.argmax(y_valid,axis=1)
y_test= np.argmax(y_test,axis=1)

txt_O = "Mean_{Class1:.0f}"
Means={}
for i in range(NB_CLASSES):
  Means[txt_O.format(Class1=i)]=np.array([0]*NB_CLASSES)

count=[0]*NB_CLASSES

for i in range(len(tflite_model_predictions)):
  k=np.argmax(tflite_model_predictions[i])
  if (np.argmax(tflite_model_predictions[i])==y_train[i]):
    Means[txt_O.format(Class1=y_train[i])]=Means[txt_O.format(Class1=y_train[i])]+tflite_model_predictions[i]
    count[y_train[i]]+=1
#print("Counts: ",count)

Mean_Vectors=[]
for i in range(NB_CLASSES):
  Means[txt_O.format(Class1=i)]=Means[txt_O.format(Class1=i)]/count[i]
  Mean_Vectors.append(Means[txt_O.format(Class1=i)])

Mean_vectors=np.array(Mean_Vectors)
np.save('/content/temp_variables/Mean_vectors.npy', Mean_vectors, allow_pickle=True)


#####################################################################################################
tflite_model_predictions = model_penultimate.predict(X_valid)

Values={}
for i in range(NB_CLASSES):
  Values[i]=[0]*NB_CLASSES

for i in range(len(tflite_model_predictions)):
  if (y_valid[i]==np.argmax(tflite_model_predictions[i])):
    dist = np.linalg.norm(Mean_Vectors[y_valid[i]]-tflite_model_predictions[i])

    for k in range(NB_CLASSES):
      if k!=int(y_valid[i]):
        Values[y_valid[i]][k]+=np.linalg.norm(Mean_Vectors[k]-tflite_model_predictions[i])-dist

Indexes=[]
for i in range(NB_CLASSES):
  Indexes.append([])

for i in range(NB_CLASSES):
  Tot=0
  for l in range(20):
      Min=min(Values[i])
      Tot+=Min
      Indexes[i].append(Values[i].index(Min))
      Values[i][Values[i].index(Min)]=1000000

Indexes=np.array(Indexes)

np.save('/content/temp_variables/Values.npy',np.array(Values))
np.save('/content/temp_variables/Indexes.npy',Indexes)


############## Calculating the 90% Thresholds for each method ##################

txt_1 = "Dist_{Class1:.0f}"
Distances={}
for i in range(NB_CLASSES):
  Distances[txt_1.format(Class1=i)]=[]


for i in range(len(tflite_model_predictions)):
  if (y_valid[i]==np.argmax(tflite_model_predictions[i])):
    dist = np.linalg.norm(Mean_Vectors[y_valid[i]]-tflite_model_predictions[i])
    Distances[txt_1.format(Class1=y_valid[i])].append(dist)

#print(Distances)
Threasholds_1=[0]*NB_CLASSES
for j in range(NB_CLASSES):
  Distances[txt_1.format(Class1=j)].sort()
  Dist=Distances[txt_1.format(Class1=j)]
  Threasholds_1[j]=Dist[int(len(Dist)*0.90)]


np.save('/content/temp_variables/Threasholds_1.npy', Threasholds_1)

print("Thresholds are calulated for method 1")

############################################################################################
txt_1 = "Dist_{Class1:.0f}"
Distances={}
for i in range(NB_CLASSES):
  Distances[txt_1.format(Class1=i)]=[]

for i in range(len(tflite_model_predictions)):
  if (y_valid[i]==np.argmax(tflite_model_predictions[i])):
    dist = np.linalg.norm(Mean_Vectors[y_valid[i]]-tflite_model_predictions[i])
    Tot=0
    for k in range(NB_CLASSES):
      if k!=int(y_valid[i]) and k in Indexes[y_valid[i]]:
        Tot+=(np.linalg.norm(Mean_Vectors[k]-tflite_model_predictions[i])-dist)
    Distances[txt_1.format(Class1=y_valid[i])].append(Tot)

#print(Distances)
TH=[0]*NB_CLASSES
for j in range(NB_CLASSES):
  Distances[txt_1.format(Class1=j)].sort()
  Dist=Distances[txt_1.format(Class1=j)]
  TH[j]=Dist[int(len(Dist)*0.12)]

Threasholds_2=np.array(TH)
np.save('/content/temp_variables/Threasholds_2.npy', Threasholds_2)

print("Thresholds are calulated for method 2")

############################################################################################
txt_1 = "Dist_{Class1:.0f}"
Distances={}
for i in range(NB_CLASSES):
  Distances[txt_1.format(Class1=i)]=[]

for i in range(len(tflite_model_predictions)):
  if (y_valid[i]==np.argmax(tflite_model_predictions[i])):
    dist = np.linalg.norm(Mean_Vectors[y_valid[i]]-tflite_model_predictions[i])
    Tot=0
    for k in range(NB_CLASSES):
      if k!=int(y_valid[i]) and k in Indexes[y_valid[i]]:
        Tot+=np.linalg.norm(Mean_Vectors[k]-tflite_model_predictions[i])

    Tot=dist/Tot
    Distances[txt_1.format(Class1=y_valid[i])].append(Tot)

#print(Distances)
TH=[0]*NB_CLASSES
for j in range(NB_CLASSES):
  Distances[txt_1.format(Class1=j)].sort()
  Dist=Distances[txt_1.format(Class1=j)]
  TH[j]=Dist[int(len(Dist)*0.88)]

Threasholds_3=np.array(TH)
np.save('/content/temp_variables/Threasholds_3.npy', Threasholds_3)

print("Thresholds are calulated for method 3")
print(Threasholds_3)

337/337 [==============================] - 2s 5ms/step
Thresholds are calulated for method 1
Thresholds are calulated for method 2
Thresholds are calulated for method 3
[0.0311762  0.0297349  0.02463876 0.03191956 0.03189763 0.02689045
 0.0299354  0.02550673 0.03021136 0.02693266 0.02532371 0.02555082
 0.03049721 0.02297539 0.02595602 0.02857334 0.02469235 0.02278963
 0.02550774 0.03256061 0.01937267 0.03075757 0.02548889 0.03057417
 0.02815366 0.02830803 0.02529855 0.03441702 0.03117788 0.0274227
 0.03073766 0.02556757 0.03169732 0.02044747 0.02641906 0.03256585
 0.04118822 0.02469965 0.02471913 0.02805142]


In [7]:
############################################################################################

######################## Evaluating results ##########################
dataset_dir="/content/gdrive/My Drive/SydneyDatasets/IoT/"
X_open = np.load(dataset_dir+'X_open.npy')
y__open = np.load(dataset_dir+'y_open.npy')
y_open=np.array([NB_CLASSES]*len(y__open))

X_open = X_open.astype('float32')
y_open = y_open.astype('int8')

X_open = X_open[:, :,np.newaxis]

tflite_model_predictions = model_penultimate.predict(X_test)
tflite_model_predictions_open = model_penultimate.predict(X_open)

#############################################################################################################
print()
print("############## Distance Method 1 #################################")
print()
prediction_classes = []
for i in range(len(tflite_model_predictions)):

    d = np.argmax(tflite_model_predictions[i], axis=0)
    if np.linalg.norm(tflite_model_predictions[i] - Mean_vectors[d]) > Threasholds_1[d]:
        prediction_classes.append(NB_CLASSES)

    else:
        prediction_classes.append(d)

prediction_classes = np.array(prediction_classes)
acc_Close = accuracy_score(prediction_classes, y_test)
print('Test accuracy Normal model_Closed_set :', acc_Close)


prediction_classes_open = []
for i in range(len(tflite_model_predictions_open)):

    d = np.argmax(tflite_model_predictions_open[i], axis=0)
    if np.linalg.norm(tflite_model_predictions_open[i] - Mean_vectors[d]) > Threasholds_1[d]:
        prediction_classes_open.append(NB_CLASSES)

    else:
        prediction_classes_open.append(d)

prediction_classes_open = np.array(prediction_classes_open)

acc_Open = accuracy_score(prediction_classes_open, y_open)
print('Test accuracy Normal model_Open_set :', acc_Open)

Matrix = []
for i in range(NB_CLASSES + 1):
    Matrix.append(np.zeros(NB_CLASSES + 1))

for i in range(len(y_test)):
    Matrix[y_test[i]][prediction_classes[i]] += 1

for i in range(len(y_open)):
    Matrix[y_open[i]][prediction_classes_open[i]] += 1

#print(Matrix)

print()
F1_Score = New_F1_Score(Matrix)
print("Average novel F1_Score: ", F1_Score)

print()
F1_Score = Micro_F1(Matrix)
print("Average Micro F1_Score: ", F1_Score)

print()
F1_Score = Macro_F1(Matrix)
print("Average Macro F1_Score: ", F1_Score)

#############################################################################################################
print()
print("############## Distance Method 2 #################################")
print()
prediction_classes=[]
for i in range(len(tflite_model_predictions)):

    d=np.argmax(tflite_model_predictions[i], axis=0)
    #dist=np.dot(Mean_vectors[d],tflite_model_predictions[i])
    dist=np.linalg.norm(Mean_vectors[d]-tflite_model_predictions[i])
    Tot=0
    for k in range(NB_CLASSES):
      if k!=d and (k in Indexes[d]):
        #Tot+=np.dot(Mean_vectors[k],tflite_model_predictions[i])-dist
        Tot+=np.linalg.norm(Mean_vectors[k]-tflite_model_predictions[i])-dist

    if Tot<Threasholds_2[d]:
      prediction_classes.append(NB_CLASSES)

    else:
      prediction_classes.append(d)

prediction_classes_open=[]
for i in range(len(tflite_model_predictions_open)):
    d=np.argmax(tflite_model_predictions_open[i], axis=0)
    #dist=np.dot(Mean_vectors[d],tflite_model_predictions_open[i])
    dist = np.linalg.norm(Mean_vectors[d]-tflite_model_predictions_open[i])
    Tot=0
    for k in range(NB_CLASSES):
      if k!=int(d) and k in Indexes[d]:
        #Tot+=np.dot(Mean_vectors[k],tflite_model_predictions_open[i])-dist
        Tot+=np.linalg.norm(Mean_vectors[k]-tflite_model_predictions_open[i])-dist

    if Tot<Threasholds_2[d]:
      prediction_classes_open.append(NB_CLASSES)

    else:
      prediction_classes_open.append(d)


acc_Close = accuracy_score(prediction_classes, y_test[:len(prediction_classes)])
print('Test accuracy TFLITE model_Closed_set :', acc_Close)

acc_Open = accuracy_score(prediction_classes_open, y_open[:len(prediction_classes_open)])
print('Test accuracy TFLITE model_Open_set :', acc_Open)

Matrix = []
for i in range(NB_CLASSES + 1):
    Matrix.append(np.zeros(NB_CLASSES + 1))

for i in range(len(y_test)):
    Matrix[y_test[i]][prediction_classes[i]] += 1

for i in range(len(y_open)):
    Matrix[y_open[i]][prediction_classes_open[i]] += 1

#print(Matrix)

print()
F1_Score = New_F1_Score(Matrix)
print("Average novel F1_Score: ", F1_Score)

print()
F1_Score = Micro_F1(Matrix)
print("Average Micro F1_Score: ", F1_Score)

print()
F1_Score = Macro_F1(Matrix)
print("Average Macro F1_Score: ", F1_Score)

#############################################################################################################
print()
print("############## Distance Method 3 #################################")
print()
prediction_classes=[]
for i in range(len(tflite_model_predictions)):

    d=np.argmax(tflite_model_predictions[i], axis=0)
    dist=np.linalg.norm(Mean_vectors[d][d]-tflite_model_predictions[i][d])
    #dist=np.dot(Mean_vectors[d],tflite_model_predictions[i])
    Tot=0
    for k in range(NB_CLASSES):
      if k!=d and (k in Indexes[d]):
        Tot+=np.linalg.norm(Mean_vectors[k]-tflite_model_predictions[i])
        #Tot+=np.dot(Mean_vectors[k],tflite_model_predictions[i])

    Tot=dist/Tot
    if Tot>Threasholds_3[d]:
      prediction_classes.append(NB_CLASSES)

    else:
      prediction_classes.append(d)

prediction_classes_open=[]
for i in range(len(tflite_model_predictions_open)):
    d=np.argmax(tflite_model_predictions_open[i], axis=0)
    dist=np.linalg.norm(Mean_vectors[d]-tflite_model_predictions_open[i])
    #dist=np.dot(Mean_vectors[d],tflite_model_predictions_open[i])
    Tot=0
    for k in range(NB_CLASSES):
      if k!=int(d) and k in Indexes[d]:
        Tot+=np.linalg.norm(Mean_vectors[k]-tflite_model_predictions_open[i])
        #Tot+=np.dot(Mean_vectors[k],tflite_model_predictions_open[i])

    Tot=dist/Tot
    if Tot>Threasholds_3[d]:
      prediction_classes_open.append(NB_CLASSES)

    else:
      prediction_classes_open.append(d)

acc_Close = accuracy_score(prediction_classes, y_test[:len(prediction_classes)])
print('Test accuracy TFLITE model_Closed_set :', acc_Close)

acc_Open = accuracy_score(prediction_classes_open, y_open[:len(prediction_classes_open)])
print('Test accuracy TFLITE model_Open_set :', acc_Open)

Matrix = []
for i in range(NB_CLASSES + 1):
    Matrix.append(np.zeros(NB_CLASSES + 1))

for i in range(len(y_test)):
    Matrix[y_test[i]][prediction_classes[i]] += 1

for i in range(len(y_open)):
    Matrix[y_open[i]][prediction_classes_open[i]] += 1

#print(Matrix)

print()
F1_Score = New_F1_Score(Matrix)
print("Average novel F1_Score: ", F1_Score)

print()
F1_Score = Micro_F1(Matrix)
print("Average Micro F1_Score: ", F1_Score)

print()
F1_Score = Macro_F1(Matrix)
print("Average Macro F1_Score: ", F1_Score)

2716/2716 [==============================] - 14s 5ms/step

############## Distance Method 1 #################################

Test accuracy Normal model_Closed_set : 0.872476681052485
Test accuracy Normal model_Open_set : 0.6776810993715904

Precision:  0.1433833683131882
Recall:  0.8747819973540342
Average novel F1_Score:  0.2463827435462807

Micro Precision:  0.1824294821412977
Micro Recall:  0.872476681052485
Average Micro F1_Score:  0.3017623237958337

Precision: 0.3358765986547234
Recall: 0.872188537723433
Average Macro F1_Score:  0.484986625621071

############## Distance Method 2 #################################

Test accuracy TFLITE model_Closed_set : 0.8553529166086593
Test accuracy TFLITE model_Open_set : 0.7750040282669245

Precision:  0.18777923899636917
Recall:  0.8579656473508138
Average novel F1_Score:  0.30812129594381654

Micro Precision:  0.2386297432710607
Micro Recall:  0.8553529166086593
Average Micro F1_Score:  0.37315517423858086

Precision: 0.40388868894878616